# REFERENCIAS

In [ ]:
#https://spoddutur.github.io/spark-notes/distribution_of_executors_cores_and_memory_for_spark_application.html

# FUNÇÕES EM SCALA PARA NORMALIZAÇÃO DE ARRAYS
#https://sparkbyexamples.com/spark/spark-flatten-nested-array-column-to-single-column/

# CALCULO DE MEMÓRIA DE EXECUTORES
#https://spoddutur.github.io/spark-notes/distribution_of_executors_cores_and_memory_for_spark_application.html

# AJUSTE DE PARTIÇÕES
#https://sparkbyexamples.com/spark/spark-repartition-vs-coalesce/

# ESTIMATIVA DE MEMÓRIA USADA PELO SPARK
#https://sparkbyexamples.com/spark/spark-difference-between-cache-and-persist/

# IMPORTS

In [ ]:

import json
import bson as bson
import string as s
import pandas as pd
import os.path,inspect, re 
import random as r
from types import SimpleNamespace
import sys
import pathlib
import time
import os
import zipfile
import concurrent.futures
from pymongo import MongoClient
import databricks.koalas as ks


from pyspark.sql.types import *

from pyspark.sql import types as T 
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime, timedelta 
import pyspark
from pyspark.sql import SQLContext  

spark.conf.set('spark.Builder.enableHiveSupport',True)
spark.conf.set("spark.sql.session.timeZone", "America/Sao_Paulo")
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled' , False)
spark.conf.set('spark.sql.caseSensitive', True)
spark.conf.set("spark.sql.shuffle.partitions", "2") 


# SPARK_TOOLS

In [ ]:


class SparkTools:
    def __init__(self, CREDITOR=None,DATABASE=None,COLLECTION=None,DIR=None):
        self.CREDITOR = CREDITOR
        self.DATABASE = DATABASE
        self.COLLECTION = COLLECTION
        self.SCHEMA = self.getSchema()
        self.URI = "URI"
        self.LIST = [] 
        self.DIR = DIR 

    
        
        
    def getSchema(self,sampleSize = 50000, mode="local",col="collection"):
        DATA_PATH = "local_path" 

        if mode =="local":
          with open(DATA_PATH) as f:
            PersonSchema = T.StructType.fromJson(json.load(f))

        else:
          DATA = spark.read\
          .format( "com.mongodb.spark.sql.DefaultSource")\
          .option('spark.mongodb.input.sampleSize', sampleSize)\
          .option("database", "database")\
          .option("spark.mongodb.collection.collection", "collection")\
          .option("badRecordsPath", "/tmp/badRecordsPath")\
          .load().schema

          DATA


          with open("schema.json", "w") as f:
            json.dump(DATA.jsonValue(), f)

          with open("schema.json") as f:
            PersonSchema = T.StructType.fromJson(json.load(f))
        dbutils.widgets.dropdown("SCHEMA_STATUS", "LOADED", ["LOADED", "NOT LOADED"])
        return PersonSchema
 
    def getCols(self):
        
        COLS = spark.read.schema(self.SCHEMA)\
                .format("com.mongodb.spark.sql.DefaultSource")\
                .option('spark.mongodb.input.database', self.DATABASE)\
                .option('spark.mongodb.input.collection', self.COLLECTION)\
                .load().columns
        return COLS
      

    #realiza queries no banco MongoDB utilizando o driver em Python
    def findByMongo(self, DATABASE="database", COLLECTION="col_person"):
      client = MongoClient(self.collection)
      db = client.get_database(DATABASE)
      query = db.get_collection(COLLECTION)
      return query

            
    def countDir(self): 
        dcFiles = dbutils.fs.ls(self.DIR)
        df = spark.createDataFrame(sc.parallelize(dcFiles))
        count = df.count()
        displayHTML("DIRECTORY SET AS: ...  %s" % self.DIR)
        displayHTML("TOTAL OF FILES FOUND AFTER SCANNING : ... %s" % count)   

    def graphDir(self):
        dcFiles = dbutils.fs.ls(self.DIR)
        df = spark.createDataFrame(sc.parallelize(dcFiles))
        df = df.sort(df.size)
        count = df.count()
        display(df)

    def descDir(self):
        #origin
        dcFiles = dbutils.fs.ls(self.dc)
        df = spark.createDataFrame(sc.parallelize(dcFiles))
        df = df.describe()
        display(df)

    def filesDir(self):
        #origin
        dcFiles = dbutils.fs.ls(self.DIR)
        return dcFiles       
      


    def getInterByIdSingle(self,id=None):
      col = st.findByMongo(COLLECTION='collection')
      col_filter = [
          {
              '$match': {
                  '_id': '%s' % id
              }
          }
                    ]
      result = col.aggregate(col_filter)
      result = list(result)
      #UTILIZAR ALGUMA FORMA DE PERSISTÊNCIA EM MEMÓRIA PARA ACUMULAR OS RESULTADOS
      self.LIST.append(result)
      size = len(self.INTER_LIST)
      print(size)

    def threadExecute(self, FUNC = None, ITERABLE=None):
      
      #THREAD POOL EXECUTOR
      with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        #ITERABLE
        for ITEM in ITERABLE:
          #FUNCTION
          futures.append(executor.submit(FUNC, PARAM=ITEM))


## PLOT COLLECTION

### FUNNEL PLOT

In [ ]:
from plotly import graph_objects as go


fig = go.Figure()
name_list = [x for x in 'ABCDE']
step_list = ["Website visit", "Downloads", "Potential customers", "Requested price", "invoice sent", "Finalized"]
step_values = [10,20,30,40,50]
textinfo = ["value+percent total"]


trace_list =    [
                    {
                        "name":name_list[0],
                        "orientation": "h",
                        "y":step_list,
                        "x": step_values,
                        "textinfo": textinfo
                    },
                    {
                        "name":name_list[1],
                        "orientation": "h",
                        "y":step_list,
                        "x": step_values,
                        "textinfo": textinfo
                    },
                    {
                        "name":name_list[2],
                        "orientation": "h",
                        "y":step_list,
                        "x": step_values,
                        "textinfo": textinfo
                    },
                    {
                        "name":name_list[3],
                        "orientation": "h",
                        "y":step_list,
                        "x": step_values,
                        "textinfo": textinfo
                    }
                ]

for x in trace_list:
    fig.add_trace(go.Funnel(
        name = x['name'],
        orientation=x['orientation'],
        y = list(x['y']),
        x = x['x'],
        textinfo = x['textinfo'][0]))
fig.show()
    


%md ALTERNATIVE PLOT

In [ ]:
import plotly.graph_objects as go
import pandas as pd


A ={"nome" : "A", "tamanho" : 12, "parents": "empresas"}
B ={"nome" : "B", "tamanho" : 5, "parents": "empresas"} 
l1 = [A, B]
df = pd.DataFrame(l1)

fig = go.Figure(
    go.Icicle(
        ids = df.nome,
        labels = df.tamanho,
        parents = df.parents,
        root_color="lightgrey",
        tiling = dict(
            orientation='h'
        )
    )
)
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

# Lambda Testing

#

%md map fund

In [7]:
 Built-In Functions
 – when()
 – expr()
 – lit()
 – split()
 – concat_ws()
–   substring()
 – translate()
 – regexp_replace()
 – overlay()
 – to_timestamp()
 – to_date()
 – date_format()
 – datediff()
 – months_between()
 – explode()
 – array_contains()
 – array()
 – collect_list()
 – collect_set()
 – create_map()
 – map_keys()
 – map_values()
 – struct()
 – countDistinct()
 – sum(), avg()
 – row_number()
 – rank()
 – dense_rank()
 – percent_rank()
 – typedLit()
 – from_json()
 – to_json()
 – json_tuple()
 – get_json_object()
 – schema_of_json()

NameError: name 'koalas' is not defined

In [17]:
import numpy as np
letters = [x for x in "abdce"]
numbers = [x for x in range(5)]
a = np.array(letters)
b = np.array(numbers)

path = 'C:\Users\QQ\Documents\interaction_ids.csv'
import pandas as pd
df = pd.read_csv(path)
df


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-17-81bf515c7309>, line 7)